# PARAFAC Decomposition Example 
### (aka CP / CANDECOMP / PARAFAC)

Идея тензорного разложения заключается в представлении исходного тензора X в виде суммы триад. Минимальное количество триад, на которые можно разложить тензор, назы- вается рангом тензора. Псевдо-ранг - это кол-во триад, которое получилось по итогу разложения (и это не всегда ранг, мы можем приблизить исходный тензор до какой-то допустимой для задачи точности, а остальное уйдет в тензор ошибок).

PARAFAC - по сути, частный случай разложения Таккера с единичным core-тензором G. И, в трехмерном случае, вместо суммы произведений маленького тензора на 3 матрицы, получается сумма произведений 3-х векторов (потому, что на супер диагонали G будут единицы).

### 3х-мерный случай:

\begin{align}
\mathcal{\underline X} = \sum_{r=1}^R a_{r} * b_{r} * c_{r} + {\underline E},
\end{align}

где $\mathcal{{\underline X} - исходный \space тензор}$,
$\mathcal{a_r, \space b_r, \space c_r - вектора}$,
$\mathcal{* - кронекерово \space произведение}$,
$\mathcal{{\underline E} - тензор \space ошибок}$,
$\mathcal{{R} - псевдо-ранг \space тензора \space (количество \space слагаемых).}$,

### D-мерный случай:
Для d-мерного тензора разложение будет выглядеть следующим образом:

$\mathcal{\underline X} \in \mathbb{R}^{d}$ :

\begin{align}
\mathcal{\underline X}_{(i_{1}, ... , i_{d})} = \sum_{r=1}^R A_{(i_{1},\space r)} * ... * A_{(i_{d},\space r)} + {\underline E},
\end{align}

In [38]:
import math
import numpy as np

# decomp_rank - кол-во слагаемых
# в соответствии с вышеуказанной формулой, мы должны вернуть: 
# разложение в виде векторов
# тензор ошибок нашего разложения

class Triades:
    def __init__(self, A, B, C, tensor_shape, row, slice):
        self.A = A
        self.B = B
        self.C = C
        self.tensor_shape = tensor_shape
        self.row = row
        self.slice = slice


def get_factor_decomp_3d(tensor, row, slice):
    A = tensor[row, :, slice] # i-я строка k-го среза
    B = np.zeros_like(tensor[:, 0, slice]) # вектор из нулей
    B[row] = 1 # установили единицу в нужной строке для умножения
    C = np.zeros_like(tensor[0, 0, :]) # вектор из нулей
    C[slice] = 1 # установили единицу в нужном срезе для умножения

    triade = Triades(A=A, B=B, C=C, tensor_shape=tensor.shape, row=row, slice=slice)
    # print(triade.A)
    return triade

def get_factor_product_3d(triade):
    factor_tensor = np.zeros(triade.tensor_shape)
    factor_tensor[triade.slice, triade.row, :] = triade.A
    return factor_tensor
    

def get_diff(tensor_1, tensor_2):
    return tensor_1 - tensor_2

def get_summ(triades_list):
    tensor_list = []
    for i in range(len(triades_list)):
        tensor_list.append(get_factor_product_3d(triades_list[i]))
    if len(tensor_list) < 1:
        return []
    summ = np.zeros_like(tensor_list[0])
    for i in range(len(tensor_list)):
        summ = summ + tensor_list[i]
    return summ


def parafac_3d(data_tensor, decomp_rank):
    I, J, K = data_tensor.shape
    factor_list = []
    error_tensor = np.zeros_like(data_tensor)
    print("data_tensor shape:", data_tensor.shape)

    if (decomp_rank):
        K = decomp_rank

    # заполняем массив факторов:
    for i in range(I):
        for k in range(K):
            factor_list.append(get_factor_decomp_3d(data_tensor, i, k))
    
    error_tensor = get_diff(data_tensor, get_summ(factor_list))

    return factor_list, error_tensor

    

Здесь возьмем простую функцию (сумма индексов), чтобы можно было визуально оценить, что мы получаем при разложении:

In [40]:
def myMatrixIndexSumFunc(i, j, k):
    return i + j + k + 3

N1 = N2 = N3 = 2
T = np.fromfunction(myMatrixIndexSumFunc, [N1,N2,N3])

# здесь нормальное разложение, по кол-ву элементов:
factor_list, error_tensor = parafac_3d(T, 2)
#print("factor_list:")
#print(factor_list)
print("error_tensor:")
print(error_tensor)
print("data_tensor:")
print(T)

# здесь кол-во слагаемых меньше, чем нужно, поэтому тензор ошибок будет ненулевым:
factor_list, error_tensor = parafac_3d(T, 1) 
#print("factor_list:")
#print(factor_list)
print("error_tensor:")
print(error_tensor)
print("data_tensor:")
print(T)

data_tensor shape: (2, 2, 2)
error_tensor:
[[[0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]]]
data_tensor:
[[[3. 4.]
  [4. 5.]]

 [[4. 5.]
  [5. 6.]]]
data_tensor shape: (2, 2, 2)
error_tensor:
[[[0. 0.]
  [0. 0.]]

 [[4. 5.]
  [5. 6.]]]
data_tensor:
[[[3. 4.]
  [4. 5.]]

 [[4. 5.]
  [5. 6.]]]


Для нормальных (не учебных) задач можно воспользоваться библиотечной реализацией parafac (их довольно много - tensorly, t3f, ) - например, из tensorly:

In [96]:
import tensorly as tl
from tensorly.decomposition import parafac

T = np.fromfunction(myMatrixIndexSumFunc, [N1,N2,N3])
test2 = parafac(T, 2)

print(test2.weights)
print(test2.factors)

[1. 1.]
[array([[ 8.5940423 , -0.24768112],
       [10.08277184, -0.14092481]]), array([[0.66508732, 1.72292692],
       [0.78026222, 0.98087872]]), array([[0.66813138, 1.93681297],
       [0.78379646, 1.10328796]])]
